A generalização empilhada é um método para combinar estimadores para reduzir seus vieses. 
Mais precisamente, as previsões de cada estimador individual são empilhadas e usadas como entrada para um estimador final para calcular a previsão. 
Este estimador final é treinado por meio de validação cruzada.

[Documentação](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.StackingClassifier.html#sklearn.ensemble.StackingClassifier)

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import StackingClassifier
from sklearn.model_selection import StratifiedShuffleSplit, train_test_split
import pandas as pd
import os
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
bases_prontas_path = os.path.join("D:\\","FIA","TCC","BASES","")
df = pd.read_csv(bases_prontas_path+'mask_dataset_vgg16_preprocess_input_224_224_3_feature_extracted.csv')
X, y = df.drop(['im_path', 'class'], axis=1), df['class'].values

label_transformer = preprocessing.LabelEncoder()
label_transformer.fit(y)
y = label_transformer.transform(y)

cross_validation = StratifiedShuffleSplit(n_splits=7, test_size=0.3, random_state=42)

In [ ]:
clf1 = SGDClassifier(alpha=0.31622776601683794, 
                     max_iter=100, 
                     random_state=42, 
                     tol=1e-06, 
                     loss="hinge", 
                     penalty = "l2")
clf2 = KNeighborsClassifier(algorithm='brute', 
                            metric='braycurtis', 
                            n_neighbors=6, 
                            weights='distance')
clf3 = KNeighborsClassifier(algorithm='brute', 
                            metric='braycurtis', 
                            n_neighbors=3, 
                            weights='distance')
clf4 = LogisticRegression(C=50, 
                          l1_ratio=0, 
                          max_iter=10000, 
                          random_state=42, 
                          solver = "liblinear", 
                          penalty= "l1", 
                          class_weight = "balanced")
clf5 = RandomForestClassifier(max_depth=1000, 
                              random_state=42, 
                              criterion= "gini", 
                              class_weight = "balanced")
clf6 = MLPClassifier(solver='lbfgs', 
                     random_state=42, 
                     max_iter=1000, 
                     learning_rate="invscaling")

estimadores_de_entrada = [('sgd', clf1), 
                          ('knn6', clf2), 
                          ('knn3', clf3), 
                          ('regLog', clf4),
                          ('RdnForest', clf5),
                          ('MLP', clf6)]

estimador_final = GradientBoostingClassifier(random_state=42)

In [ ]:
clf = StackingClassifier(estimators=estimadores_de_entrada, 
                         cv=cross_validation, 
                         final_estimator=estimador_final)

In [ ]:
historico = clf.fit(X_train, y_train)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.50, random_state=42)

clf.score(X_test, y_test)